# Import libraries

In [1]:
import os
import warnings

import numpy as np
import pandas as pd

import json

import re

from tqdm import tqdm

from itertools import combinations

In [2]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [4]:
path_to_save = './../results/03a_features'

# Loading data

In [5]:
path_to_ftir_data= './../data/processed_data/ftir_extracted_features.csv'
path_to_samples_description = './../data/parsed_data/samples_description.csv'

## FTIR-features

In [6]:
initial_features_df = pd.read_csv(path_to_ftir_data, sep=';', index_col=0)

print(initial_features_df.shape)
initial_features_df.head(3)

(75, 95)


,$\overline{I}_{3800\text{–}2200}$,$\overline{I}_{3800\text{–}2400}$,$I_{3720}$,$\overline{I}_{3720 \pm 25}$,$\overline{I}_{3720 \pm 50}$,$I_{3533}$,$\overline{I}_{3533 \pm 25}$,$\overline{I}_{3533 \pm 50}$,$I_{3405}$,$\overline{I}_{3405 \pm 25}$,$\overline{I}_{3405 \pm 50}$,$I_{3302}$,$\overline{I}_{3302 \pm 25}$,$\overline{I}_{3302 \pm 50}$,$I_{3201}$,$\overline{I}_{3201 \pm 25}$,$\overline{I}_{3201 \pm 50}$,$I_{3132}$,$\overline{I}_{3132 \pm 25}$,$\overline{I}_{3132 \pm 50}$,$I_{3001}$,$\overline{I}_{3001 \pm 25}$,$\overline{I}_{3001 \pm 50}$,$I_{2926}$,$\overline{I}_{2926 \pm 25}$,...,$I_{1237}$,$\overline{I}_{1237 \pm 25}$,$\overline{I}_{1237 \pm 50}$,$I_{1180}$,$\overline{I}_{1180 \pm 25}$,$\overline{I}_{1180 \pm 50}$,$I_{1120}$,$\overline{I}_{1120 \pm 25}$,$\overline{I}_{1120 \pm 50}$,$I_{1077}$,$\overline{I}_{1077 \pm 25}$,$\overline{I}_{1077 \pm 50}$,$I_{1046}$,$\overline{I}_{1046 \pm 25}$,$\overline{I}_{1046 \pm 50}$,$I_{881}$,$\overline{I}_{881 \pm 25}$,$\overline{I}_{881 \pm 50}$,"$\Sigma I_{p,\ 3800\text{–}2200}$","$\Sigma I_{p,\ 3800\text{–}2400}$","$\Sigma I_{p,\ 2400\text{–}2200}$","$\Sigma I_{p,\ 1800\text{–}900}$","$\Sigma I_{p,\ 1800\text{–}1500}$","$\Sigma I_{p,\ 1500\text{–}1300}$","$\Sigma I_{p,\ 1300\text{–}900}$"
division_1_size_bulk,0.425422,0.462285,0.084601,0.076032,0.071260,0.506784,0.511390,0.524201,0.938564,0.935451,0.928330,1.000000,0.967122,0.963302,0.815223,0.819056,0.822279,0.720771,0.714901,0.710538,0.498252,0.499682,0.501630,0.422644,0.422996,...,0.550066,0.544341,0.542669,0.515502,0.526407,0.554926,0.745328,0.737048,0.722452,0.830578,0.825993,0.808403,0.846152,0.827226,0.792935,0.279657,0.280558,0.284044,5.716254,5.349151,0.367102,9.204488,4.214849,1.502012,3.487627
division_1_size_5,0.439080,0.480354,0.038211,0.036371,0.046173,0.652508,0.657052,0.667775,1.000000,0.985547,0.976453,0.942571,0.941360,0.940593,0.814723,0.815564,0.814640,0.704920,0.702329,0.701261,0.486172,0.487186,0.494274,0.448983,0.445307,...,0.508943,0.507598,0.506955,0.515948,0.524999,0.544120,0.705649,0.682573,0.656617,0.664996,0.655746,0.633505,0.553350,0.552197,0.551724,0.249361,0.250764,0.255095,5.763332,5.448649,0.314682,8.405115,4.096314,1.359915,2.948886
division_1_size_3,0.428649,0.463028,0.074497,0.065544,0.062297,0.598526,0.602768,0.611808,1.000000,0.959284,0.951557,0.937402,0.935431,0.933702,0.760651,0.763172,0.764610,0.637583,0.636283,0.638100,0.450708,0.455257,0.473817,0.547936,0.532374,...,0.475526,0.478748,0.490063,0.618890,0.631078,0.651870,0.953037,0.926203,0.879436,0.879241,0.871894,0.850851,0.746178,0.743572,0.739470,0.304707,0.304898,0.304952,5.778349,5.347866,0.430483,9.432362,4.374994,1.384496,3.672872


## Samples description

In [7]:
samples_description = pd.read_csv(f'{path_to_samples_description}', sep=';', index_col=0)

print(samples_description.shape)
samples_description.head(3)

(75, 6)


,Row_ID,Division,Size,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,1,1,bulk,$> 0$,$d > 5$,2
division_1_size_5,2,1,5,$< 5$,$2 < d \leq 5$,1
division_1_size_3,3,1,3,$< 3$,$2 < d \leq 5$,1


# Feature engineering

## Function for processing

In [8]:
def generate_transformed_features(data, feature_list, tolerance=50):
    data_transformed = pd.DataFrame(index=data.index)

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def wrap_log(col):
        return fr"$\log {clean(col)}$"

    def wrap_pow(col, power):
        return fr"$({clean(col)})^{{{power}}}$"

    def wrap_div(col1, col2):
        return fr"$\frac{{{clean(col1)}}}{{{clean(col2)}}}$"

    def extract_center(name):
        nums = re.findall(r'\d+', name)
        return int(nums[0]) if nums else None

    def get_interval_tag(center):
        if center is None:
            return None
        if 2200 <= center <= 3800:
            return "high"
        elif 800 <= center <= 1900:
            return "low"
        return None

    def extract_center_and_type(name):
        center = extract_center(name)
        is_avg = r'\overline{I}' in name or '±' in name
        return center, is_avg

    def is_near_duplicate(f1, f2):
        c1, avg1 = extract_center_and_type(f1)
        c2, avg2 = extract_center_and_type(f2)
        if c1 is None or c2 is None:
            return False
        return abs(c1 - c2) <= tolerance and (avg1 or avg2)

    np.seterr(divide='ignore', invalid='ignore')

    for col in feature_list:
        base = data[col]
        data_transformed[wrap_log(col)] = np.log(base.replace(0, np.nan))
        for power in [-2, -1, 2]:
            transformed = np.power(base, power)
            transformed.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_transformed[wrap_pow(col, power)] = transformed

    for f1, f2 in combinations(feature_list, 2):
        if is_near_duplicate(f1, f2):
            continue
        c1 = extract_center(f1)
        c2 = extract_center(f2)
        tag1 = get_interval_tag(c1)
        tag2 = get_interval_tag(c2)
        if tag1 != tag2:
            continue

        ratio = data[f1] / data[f2]
        ratio.replace([np.inf, -np.inf], np.nan, inplace=True)
        data_transformed[wrap_div(f1, f2)] = ratio

    return data_transformed

## Calculation process

In [9]:
engineered_features_df = generate_transformed_features(data=initial_features_df, feature_list=initial_features_df.columns, tolerance=50)

print(engineered_features_df.shape)
engineered_features_df.head(3)

(75, 2456)


,$\log \overline{I}_{3800\text{–}2200}$,$(\overline{I}_{3800\text{–}2200})^{-2}$,$(\overline{I}_{3800\text{–}2200})^{-1}$,$(\overline{I}_{3800\text{–}2200})^{2}$,$\log \overline{I}_{3800\text{–}2400}$,$(\overline{I}_{3800\text{–}2400})^{-2}$,$(\overline{I}_{3800\text{–}2400})^{-1}$,$(\overline{I}_{3800\text{–}2400})^{2}$,$\log I_{3720}$,$(I_{3720})^{-2}$,$(I_{3720})^{-1}$,$(I_{3720})^{2}$,$\log \overline{I}_{3720 \pm 25}$,$(\overline{I}_{3720 \pm 25})^{-2}$,$(\overline{I}_{3720 \pm 25})^{-1}$,$(\overline{I}_{3720 \pm 25})^{2}$,$\log \overline{I}_{3720 \pm 50}$,$(\overline{I}_{3720 \pm 50})^{-2}$,$(\overline{I}_{3720 \pm 50})^{-1}$,$(\overline{I}_{3720 \pm 50})^{2}$,$\log I_{3533}$,$(I_{3533})^{-2}$,$(I_{3533})^{-1}$,$(I_{3533})^{2}$,$\log \overline{I}_{3533 \pm 25}$,...,"$\frac{\overline{I}_{1046 \pm 50}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1046 \pm 50}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1046 \pm 50}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1046 \pm 50}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{I_{881}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{881}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{881}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{881}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\overline{I}_{881 \pm 25}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{881 \pm 25}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{881 \pm 25}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{881 \pm 25}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\overline{I}_{881 \pm 50}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{881 \pm 50}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{881 \pm 50}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{881 \pm 50}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 3800\text{–}2200}}{\Sigma I_{p,\ 3800\text{–}2400}}$","$\frac{\Sigma I_{p,\ 3800\text{–}2200}}{\Sigma I_{p,\ 2400\text{–}2200}}$","$\frac{\Sigma I_{p,\ 3800\text{–}2400}}{\Sigma I_{p,\ 2400\text{–}2200}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1800\text{–}1500}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\Sigma I_{p,\ 1800\text{–}1500}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1500\text{–}1300}}{\Sigma I_{p,\ 1300\text{–}900}}$"
division_1_size_bulk,-0.854673,5.525349,2.350606,0.180984,-0.771574,4.679293,2.163167,0.213707,-2.469803,139.715275,11.820122,0.007157,-2.576601,172.984355,13.152352,0.005781,-2.641413,196.925741,14.033023,0.005078,-0.679671,3.893629,1.973228,0.256830,-0.670622,...,0.086147,0.188129,0.527915,0.227357,0.030383,0.066351,0.186189,0.080186,0.030481,0.066564,0.186788,0.080444,0.030859,0.067391,0.189109,0.081443,1.068628,15.571275,14.571275,2.183824,6.128105,2.639183,2.806135,1.208515,0.430669
division_1_size_5,-0.823074,5.186957,2.277489,0.192791,-0.733232,4.333884,2.081798,0.230740,-3.264621,684.878748,26.170188,0.001460,-3.313974,755.929278,27.494168,0.001323,-3.075364,469.058277,21.657753,0.002132,-0.426932,2.348706,1.532549,0.425766,-0.419992,...,0.065641,0.134688,0.405704,0.187096,0.029668,0.060874,0.183365,0.084561,0.029835,0.061217,0.184397,0.085037,0.030350,0.062274,0.187581,0.086505,1.057754,18.314756,17.314756,2.051873,6.180616,2.850268,3.012183,1.389106,0.461163
division_1_size_3,-0.847117,5.442476,2.332911,0.183740,-0.769967,4.664285,2.159695,0.214395,-2.597003,180.188987,13.423449,0.005550,-2.725034,232.773955,15.256931,0.004296,-2.775849,257.674460,16.052242,0.003881,-0.513285,2.791477,1.670771,0.358233,-0.506223,...,0.078397,0.169022,0.534108,0.201333,0.032304,0.069647,0.220085,0.082961,0.032325,0.069691,0.220223,0.083013,0.032330,0.069703,0.220262,0.083028,1.080496,13.422929,12.422929,2.155971,6.812848,2.568116,3.159990,1.191164,0.376952


## Merging process

In [10]:
features_df = pd.merge(
    left=initial_features_df,
    right=engineered_features_df,
    how='inner',
    left_index=True,
    right_index=True
)

print(features_df.shape)
features_df.head(3)

(75, 2551)


,$\overline{I}_{3800\text{–}2200}$,$\overline{I}_{3800\text{–}2400}$,$I_{3720}$,$\overline{I}_{3720 \pm 25}$,$\overline{I}_{3720 \pm 50}$,$I_{3533}$,$\overline{I}_{3533 \pm 25}$,$\overline{I}_{3533 \pm 50}$,$I_{3405}$,$\overline{I}_{3405 \pm 25}$,$\overline{I}_{3405 \pm 50}$,$I_{3302}$,$\overline{I}_{3302 \pm 25}$,$\overline{I}_{3302 \pm 50}$,$I_{3201}$,$\overline{I}_{3201 \pm 25}$,$\overline{I}_{3201 \pm 50}$,$I_{3132}$,$\overline{I}_{3132 \pm 25}$,$\overline{I}_{3132 \pm 50}$,$I_{3001}$,$\overline{I}_{3001 \pm 25}$,$\overline{I}_{3001 \pm 50}$,$I_{2926}$,$\overline{I}_{2926 \pm 25}$,...,"$\frac{\overline{I}_{1046 \pm 50}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1046 \pm 50}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1046 \pm 50}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1046 \pm 50}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{I_{881}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{881}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{881}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{881}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\overline{I}_{881 \pm 25}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{881 \pm 25}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{881 \pm 25}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{881 \pm 25}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\overline{I}_{881 \pm 50}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{881 \pm 50}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{881 \pm 50}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{881 \pm 50}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 3800\text{–}2200}}{\Sigma I_{p,\ 3800\text{–}2400}}$","$\frac{\Sigma I_{p,\ 3800\text{–}2200}}{\Sigma I_{p,\ 2400\text{–}2200}}$","$\frac{\Sigma I_{p,\ 3800\text{–}2400}}{\Sigma I_{p,\ 2400\text{–}2200}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1800\text{–}1500}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\Sigma I_{p,\ 1800\text{–}1500}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1500\text{–}1300}}{\Sigma I_{p,\ 1300\text{–}900}}$"
division_1_size_bulk,0.425422,0.462285,0.084601,0.076032,0.071260,0.506784,0.511390,0.524201,0.938564,0.935451,0.928330,1.000000,0.967122,0.963302,0.815223,0.819056,0.822279,0.720771,0.714901,0.710538,0.498252,0.499682,0.501630,0.422644,0.422996,...,0.086147,0.188129,0.527915,0.227357,0.030383,0.066351,0.186189,0.080186,0.030481,0.066564,0.186788,0.080444,0.030859,0.067391,0.189109,0.081443,1.068628,15.571275,14.571275,2.183824,6.128105,2.639183,2.806135,1.208515,0.430669
division_1_size_5,0.439080,0.480354,0.038211,0.036371,0.046173,0.652508,0.657052,0.667775,1.000000,0.985547,0.976453,0.942571,0.941360,0.940593,0.814723,0.815564,0.814640,0.704920,0.702329,0.701261,0.486172,0.487186,0.494274,0.448983,0.445307,...,0.065641,0.134688,0.405704,0.187096,0.029668,0.060874,0.183365,0.084561,0.029835,0.061217,0.184397,0.085037,0.030350,0.062274,0.187581,0.086505,1.057754,18.314756,17.314756,2.051873,6.180616,2.850268,3.012183,1.389106,0.461163
division_1_size_3,0.428649,0.463028,0.074497,0.065544,0.062297,0.598526,0.602768,0.611808,1.000000,0.959284,0.951557,0.937402,0.935431,0.933702,0.760651,0.763172,0.764610,0.637583,0.636283,0.638100,0.450708,0.455257,0.473817,0.547936,0.532374,...,0.078397,0.169022,0.534108,0.201333,0.032304,0.069647,0.220085,0.082961,0.032325,0.069691,0.220223,0.083013,0.032330,0.069703,0.220262,0.083028,1.080496,13.422929,12.422929,2.155971,6.812848,2.568116,3.159990,1.191164,0.376952


# Creating features description

## Functions for processing

In [11]:
def classify_feature_type(base_columns, columns):
    base_columns = [col for col in base_columns if "Unnamed" not in col]
    columns = [col for col in columns if "Unnamed" not in col]

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def is_interval(col):
        col = clean(col)
        return r"\overline{I}_{" in col or r"\Sigma I_{p," in col

    def is_peak(col):
        return not is_interval(col)

    def extract_numbers(col):
        return list(map(int, re.findall(r"\d{3,4}", col)))

    def is_in_range(col, low, high):
        return any(low <= num <= high for num in extract_numbers(clean(col)))

    def classify(col, base=True):
        col_clean = clean(col)
        source = "intervals" if is_interval(col_clean) else "peaks"

        if is_in_range(col_clean, 2200, 3800):
            region = "3800_2200"
        elif is_in_range(col_clean, 800, 1900):
            region = "1900_800"
        else:
            region = "unknown"

        kind = "base" if base else "engineered"
        return f"{kind}_features_{source}_{region}"

    groups = {}

    for col in base_columns:
        key = classify(col, base=True)
        groups.setdefault(key, []).append(col)

    for col in columns:
        if col not in base_columns:
            key = classify(col, base=False)
            groups.setdefault(key, []).append(col)

    groups["base_features"] = base_columns
    groups["engineered_features"] = [col for col in columns if col not in base_columns]

    return groups

## Feature groups selection

In [12]:
base_columns = initial_features_df.columns.tolist()
columns = features_df.columns.tolist()

In [13]:
features_description_dict = classify_feature_type(base_columns=base_columns, columns=columns)

In [14]:
for feature_type, feature_lst in features_description_dict.items():
    print(f'(*) {feature_type}: {len(feature_lst)}')

(*) base_features_intervals_3800_2200: 30
(*) base_features_peaks_3800_2200: 12
(*) base_features_intervals_1900_800: 38
(*) base_features_peaks_1900_800: 15
(*) engineered_features_intervals_3800_2200: 873
(*) engineered_features_peaks_3800_2200: 114
(*) engineered_features_intervals_1900_800: 1304
(*) engineered_features_peaks_1900_800: 165
(*) base_features: 95
(*) engineered_features: 2456


# Saving data

In [15]:
features_df.to_csv('./../data/processed_data/ftir_features_long_list.csv', sep=';')

In [16]:
with open('./../data/processed_data/features_description.json', 'w', encoding='utf-8') as f:
    json.dump(features_description_dict, f, ensure_ascii=False, indent=2)